In [1]:
import pandas as pd
import numpy as np
from itertools import product 
import itertools
from datetime import datetime

from source_code.core.hypothesis_evaluation.test_handler import baseline_test_groups
import source_code.core.hypothesis_evaluation.generate_results as gen_results
from source_code.core.hypothesis_evaluation.generate_graphs import *

from source_code.models.models import get_data
from source_code.utils.tools import add_index
import sys
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
dataset = 'MovieLens'
alphas = 0.05
support = 15
dimension = 'rating'
grouping_type = [['genre']]
grouping_value = [["Western"]]
agg_type = 'anova'
test_args = ['Anova']
periods_start = [datetime(2000, 10, 1, 0, 0), datetime(2000, 11, 1, 0, 0), datetime(2000, 12, 1, 0, 0)]
period_arg = [0,18]
top_n = [-1]
period_type = 'time'

d = {0:'Traditional',1:'Coverage_Two_Sides',2:'Coverage_Side_1',3:'Coverage_Side_2', 4:'Coverage_alpha_investing'}

r = 'R2'

In [3]:
dataset = 'MovieLens'
alphas = 0.05
support = 10
dimension = 'rating'
grouping_type = [['gender'], ['gender']]
grouping_value = [['M'], ['M']]
agg_type = 'mean'
test_args = ['Paired']
periods_start = [datetime(2001, 1, 1, 0, 0), datetime(2001, 3, 1, 0, 0)]
period_arg = [3]
top_n=[-1,25,20,15,10,5]
period_type = 'time'

d = {0:'Traditional',1:'Coverage_Two_Sides',2:'Coverage_Side_1',3:'Coverage_Side_2', 4:'Coverage_alpha_investing'}

r = 'R3'

In [7]:
fdr_power = []
cov_p_values = []
cov_samples = []
time_samples = []
time_n = []

for k in range(5):
    print(f'--------{k}--------')
    
    stats, res, p_values = baseline_test_groups(top_n, dataset,[0,1,2,3,4], period_type, periods_start, period_arg, grouping_type,\
                                            grouping_value, agg_type, dimension, test_args, support, alphas, verbose=True)
    if stats is None:
        print('No Results')
    else:
        samples = stats.keys()
        
        global_stats, global_res = gen_results.get_results_csv(stats, res, top_n, samples)
        
        global_stats.to_csv(f'../experiments/{dataset}/{r}/global_stats_{k+1}.csv',index=False)
        global_res.to_csv(f'../experiments/{dataset}/{r}/global_res_{k+1}.csv',index=False)

In [5]:
parameters = f' dataset = {dataset}\n alphas = {alphas}\n support = {support}\n dimension = {dimension}\n'
parameters = parameters + f' grouping_type = {grouping_type}\n grouping_value = {grouping_value}\n'
parameters = parameters + f' agg_type = {agg_type}\n test_args = {test_args}\n periods_start = {periods_start}\n'
parameters = parameters + f' period_arg = {period_arg}\n top_n={top_n}\n period_type={period_type}'

p_values.to_csv(f'../experiments/{dataset}/{r}/p_values.csv',index=False)

fil = open(f'../experiments/{dataset}/{r}/parameters','w')
fil.write(parameters)
fil.close()

In [6]:
global_stats = []
global_res = []

for i in range(0,5):
    global_stats.append( pd.read_csv(f'../experiments/{dataset}/{r}/global_stats_{i}.csv'))
    global_res.append( pd.read_csv(f'../experiments/{dataset}/{r}/global_res_{i}.csv'))

FDR - POWER

In [7]:
samples = [10,30,50,70,100]
fdr = get_fdr_power(global_stats,global_res,samples,threshold=-1)
fdr_20 = get_fdr_power(global_stats,global_res,samples,threshold=20)

Y : Sum Coverage (Sum p-values), X : Steps 

In [8]:
samples = [100]
cov_p_n = get_cov_p_values_n(global_res,samples,threshold=20)

X : Samples , Y : sum Coverage

In [9]:
samples = [10,20,30,40,50,60,70,80,90,100]
threshold = [20,-1]
cov_sample = get_cov_samples(global_res,samples,threshold)

X : Samples , Y : Time

In [10]:
samples = [10,20,30,40,50,60,70,80,90,100]
threshold = [10,-1]
time_sample = get_time_sample(global_stats,samples,threshold)

X : n, Y : Time

In [11]:
samples = [40,100]
time_n = get_tim_n(global_stats,samples)